# Stripe Financial Connections Setup

This notebook helps you:
1. Test your Stripe API connection
2. Create a Financial Connections session
3. Connect test bank accounts
4. Verify everything works

Run this first before using the Money Manager!

## Step 1: Load API Keys

## Step 0: Install Dependencies

First, let's make sure all required packages are installed.

In [ ]:
%pip install -r ../requirements.txt > /dev/null
print("✅ Dependencies installed!")
import os
import stripe
from dotenv import load_dotenv

# Load environment variables
load_dotenv("../secrets.env")
load_dotenv("../variables.env")

# Get the Stripe API key
stripe_key = os.getenv("STRIPE_API_KEY")

if not stripe_key:
    print("❌ STRIPE_API_KEY not found in environment variables")
    print("   Add it to backend/agent/secrets.env")
    print("\nFormat:")
    print("STRIPE_API_KEY=sk_test_YOUR_KEY_HERE")
else:
    if not stripe_key.startswith("sk_test_"):
        print("⚠️  Warning: You're not using a test key!")
        print("   Test keys start with 'sk_test_'")
    else:
        stripe.api_key = stripe_key
        print("✅ Stripe API key loaded successfully!")
        print(f"   Key: {stripe_key[:20]}...")

In [ ]:
print("="*60)
print("Creating Financial Connections Session...")
print("="*60)

try:
    # First, create a test customer
    print("Creating test customer...")
    customer = stripe.Customer.create(
        name="Test User",
        email="test@example.com",
    )
    print(f"✅ Customer created: {customer.id}")
    
    # Now create the Financial Connections session
    print("\nCreating Financial Connections session...")
    session = stripe.financial_connections.Session.create(
        account_holder={
            "type": "customer",
            "customer": customer.id,
        },
        permissions=["balances", "transactions", "payment_method"],
        filters={"countries": ["US"]},
    )

    print(f"\n✅ Session created successfully!")
    print(f"\n" + "="*60)
    print("IMPORTANT: SAVE THIS SESSION ID")
    print("="*60)
    print(f"\n{session.id}")
    print("\n" + "="*60)
    
    # Save to a variable for later use
    STRIPE_SESSION_ID = session.id
    
    # Check if session has client_secret for URL generation
    if hasattr(session, 'client_secret') and session.client_secret:
        auth_url = f"https://connect.stripe.com/setup/s/{session.client_secret}"
        print(f"\n🔗 To connect test bank accounts, open this URL:")
        print(f"\n{auth_url}")
    else:
        print(f"\n📋 Session created! You can now use this session ID: {session.id}")
        print(f"\nTo connect accounts, you'll need to use Stripe's hosted auth flow.")
        print(f"Check the session object for the authorization URL.")
    
    print(f"\n📝 When prompted, use these test credentials:")
    print(f"   Bank: Search for 'Test Institution'")
    print(f"   Username: user_good")
    print(f"   Password: pass_good")
    print(f"\n💡 Stripe will give you fake checking and savings accounts to test with!")
    print(f"\n⚠️  After connecting accounts in the browser, come back here and run the next cell.")
    
    # Debug: print all session attributes
    print(f"\n🔍 Debug - Session attributes:")
    print(f"   {dir(session)}")

except stripe.error.StripeError as e:
    print(f"\n❌ Error creating session: {e}")
    print("\nMake sure:")
    print("1. Your Stripe API key is correct")
    print("2. You're using a test key (starts with sk_test_)")
    print("3. Financial Connections is enabled in your Stripe dashboard")

## Step 2: Create Financial Connections Session

This creates a session that lets you connect test bank accounts.

**Only run this once!** Save the session ID it gives you.

In [ ]:
print("="*60)
print("Creating Financial Connections Session...")
print("="*60)

try:
    session = stripe.financial_connections.Session.create(
        account_holder={
            "type": "individual",
        },
        permissions=["balances", "transactions", "payment_method"],
        filters={"countries": ["US"]},
    )

    print(f"\n✅ Session created successfully!")
    print(f"\n" + "="*60)
    print("IMPORTANT: SAVE THIS SESSION ID")
    print("="*60)
    print(f"\n{session.id}")
    print("\n" + "="*60)
    
    # Save to a variable for later use
    STRIPE_SESSION_ID = session.id
    
    print(f"\n🔗 To connect test bank accounts, open this URL:")
    print(f"\n{session.url}")
    print(f"\n📝 When prompted, use these test credentials:")
    print(f"   Bank: Search for 'Test Institution'")
    print(f"   Username: user_good")
    print(f"   Password: pass_good")
    print(f"\n💡 Stripe will give you fake checking and savings accounts to test with!")
    print(f"\n⚠️  After connecting accounts in the browser, come back here and run the next cell.")

except stripe.error.StripeError as e:
    print(f"\n❌ Error creating session: {e}")
    print("\nMake sure:")
    print("1. Your Stripe API key is correct")
    print("2. You're using a test key (starts with sk_test_)")
    print("3. Financial Connections is enabled in your Stripe dashboard")

## Step 3: Paste Your Session ID

If you already created a session earlier, paste the session ID here.

Otherwise, copy it from the cell above!

In [ ]:
# If you already have a session ID, paste it here
# Otherwise, it's already set from the cell above

# STRIPE_SESSION_ID = "fcsess_..."  # Uncomment and paste your session ID if needed

print(f"Using session ID: {STRIPE_SESSION_ID[:30]}...")

## Step 4: Check Connected Accounts

After you've opened the URL above and connected test bank accounts,
run this cell to verify they're connected!

In [ ]:
print("="*60)
print("Checking Connected Accounts...")
print("="*60)

try:
    session = stripe.financial_connections.Session.retrieve(STRIPE_SESSION_ID)

    if not session.accounts or len(session.accounts.data) == 0:
        print("\n⚠️  No accounts connected yet")
        print("   Open the URL from Step 2 and connect test accounts first!")
    else:
        print(f"\n✅ Found {len(session.accounts.data)} connected accounts:\n")

        for account in session.accounts.data:
            # Get full account details
            full_account = stripe.financial_connections.Account.retrieve(account.id)

            balance = "N/A"
            if hasattr(full_account, 'balance') and full_account.balance:
                balance = f"${full_account.balance.current / 100:.2f}"

            print(f"  📊 {full_account.display_name or 'Account'}")
            print(f"     Type: {full_account.subcategory}")
            print(f"     Balance: {balance}")
            print(f"     Last 4: {full_account.last4}")
            print(f"     ID: {full_account.id}")
            print()
        
        print("\n✅ All set! Your test bank accounts are connected.")
        print("\n" + "="*60)
        print("NEXT STEP: Save this to your secrets.env file:")
        print("="*60)
        print(f"\nSTRIPE_SESSION_ID={STRIPE_SESSION_ID}")
        print("\nThen you can use the Money Manager demo!")

except stripe.error.StripeError as e:
    print(f"\n❌ Error retrieving accounts: {e}")
    print("\nMake sure your session ID is correct.")

## Step 7: Test the Stripe Client with Real Balances

Now let's test if our Stripe client can correctly retrieve the balances!

In [ ]:
# Import the Stripe client
import sys
sys.path.append('..')

from stripe_integration.client import StripeFinancialClient

# Create client
client = StripeFinancialClient()

print("="*60)
print("Testing Stripe Financial Client")
print("="*60)

# Get accounts
print("\nFetching accounts with balances...\n")

accounts = await client.get_accounts(STRIPE_SESSION_ID)

if accounts:
    print("✅ Stripe integration working!\n")
    print("Your accounts:")
    print("-" * 60)
    
    total = 0
    for acc in accounts:
        print(f"\n💰 {acc.name}")
        print(f"   Type: {acc.type.title()}")
        print(f"   Balance: ${acc.balance:,.2f}")
        print(f"   Institution: {acc.institution_name}")
        print(f"   Last 4: {acc.last_four}")
        total += acc.balance
    
    print("\n" + "=" * 60)
    print(f"Total across all accounts: ${total:,.2f}")
    print("=" * 60)
    
    print("\n✅ Perfect! Your real Stripe test accounts are working!")
    print("\n📝 Save this to secrets.env:")
    print(f"   STRIPE_SESSION_ID={STRIPE_SESSION_ID}")
    print("\nNow you can use money_manager_demo.ipynb!")
else:
    print("❌ Couldn't get accounts. Check your session ID.")

In [ ]:
# Print ALL account attributes to debug
print("="*60)
print("Complete Account Data Dump")
print("="*60)

try:
    session = stripe.financial_connections.Session.retrieve(STRIPE_SESSION_ID)
    
    for idx, account in enumerate(session.accounts.data, 1):
        account_id = account.id
        print(f"\n{'='*60}")
        print(f"ACCOUNT {idx}: {account_id}")
        print(f"{'='*60}")
        
        # Get full account details
        full_account = stripe.financial_connections.Account.retrieve(account_id)
        
        # Print the raw JSON representation
        print("\n📋 RAW ACCOUNT DATA (as dictionary):")
        print("-" * 60)
        
        # Convert to dict and print nicely
        account_dict = full_account.to_dict()
        import json
        print(json.dumps(account_dict, indent=2))
        
        print("\n" + "-" * 60)
        print("\n🔍 PYTHON ATTRIBUTES:")
        print("-" * 60)
        
        # Print all attributes
        for attr in dir(full_account):
            if not attr.startswith('_'):  # Skip private attributes
                try:
                    value = getattr(full_account, attr)
                    if not callable(value):  # Skip methods
                        print(f"{attr}: {value}")
                except:
                    pass
        
        print("\n" + "="*60)
    
    print("\n✅ Data dump complete!")
    print("\nLook for 'balance' or 'balance_refresh' in the output above")
    
except Exception as e:
    print(f"\n❌ Error: {e}")
    import traceback
    traceback.print_exc()

## Step 6: Refresh Account Balances

Now let's request fresh balance data from the test accounts!

In [ ]:
print("="*60)
print("Refreshing Account Balances...")
print("="*60)

try:
    session = stripe.financial_connections.Session.retrieve(STRIPE_SESSION_ID)
    
    for idx, account in enumerate(session.accounts.data, 1):
        account_id = account.id
        print(f"\n📊 Account {idx}: {account_id}")
        
        # Request a balance refresh
        print("   Requesting balance refresh...")
        try:
            refresh = stripe.financial_connections.Account.refresh_account(
                account_id,
                features=['balance']
            )
            
            print(f"   ✅ Balance refresh requested!")
            
            # Get updated account info
            updated_account = stripe.financial_connections.Account.retrieve(account_id)
            
            print(f"\n   Name: {updated_account.display_name}")
            print(f"   Type: {updated_account.subcategory}")
            
            # Check balance
            if hasattr(updated_account, 'balance') and updated_account.balance:
                balance_data = updated_account.balance
                if hasattr(balance_data, 'current'):
                    balance = balance_data.current / 100
                    print(f"   💰 Balance: ${balance:,.2f}")
                else:
                    print(f"   Balance data: {balance_data}")
            else:
                print(f"   ⚠️  Balance still not available")
                
        except Exception as refresh_error:
            print(f"   ❌ Balance refresh failed: {refresh_error}")
            print(f"   (This is common with test accounts - they may not support balance refresh)")
    
    print("\n" + "="*60)
    print("Balance refresh attempt complete!")
    print("="*60)
    
except Exception as e:
    print(f"\n❌ Error: {e}")
    import traceback
    traceback.print_exc()